check out gephi.cql and behaviour_desktop.md

/media/lnr-ai/applications/./neo4j-desktop-offline-1.2.7-x86_64.AppImage

In [48]:
import pandas as pd
from py2neo import Graph
graph = Graph("bolt://localhost:7687", user='neo4j', password='newPassword')
import time
import glob
import os

#### Is the Neo4J db up and running?

In [4]:
graph.run("CALL db.schema.visualization()").data()

[{'nodes': [(_-4:Company {constraints: ['CONSTRAINT ON ( company:Company ) ASSERT (company.companyname) IS UNIQUE'], indexes: [], name: 'Company'}),
   (_-2:Merchant {constraints: ['CONSTRAINT ON ( merchant:Merchant ) ASSERT (merchant.franchisename) IS UNIQUE'], indexes: [], name: 'Merchant'}),
   (_-1:Client {constraints: ['CONSTRAINT ON ( client:Client ) ASSERT (client.dedupestatic) IS UNIQUE'], indexes: [], name: 'Client'}),
   (_-3:Segment {constraints: ['CONSTRAINT ON ( segment:Segment ) ASSERT (segment.seg_l3_num) IS UNIQUE'], indexes: [], name: 'Segment'})],
  'relationships': [(Client)-[:CLIENT_LINK_7916 {}]->(Client),
   (Client)-[:TRANSACTED_AT {}]->(Merchant),
   (Client)-[:CLIENT_LINK_2697 {}]->(Client),
   (Client)-[:CLIENT_LINK {}]->(Client),
   (Client)-[:COMPANY_TRANSACTED_AT {}]->(Company),
   (Company)-[:COMPANY_LINK {}]->(Company),
   (Merchant)-[:MERCHANT_FEET_LINK_45500 {}]->(Merchant),
   (Merchant)-[:MERCHANT_VALUE_LINK {}]->(Merchant),
   (Merchant)-[:MERCHANT_L

In [5]:
df=pd.read_csv(filepath_or_buffer='/media/lnr-ai/christo/github_repos/graphing/data/nedbank_companies_df.csv')

In [6]:
gf=df[df.subclass_id==56101].copy()

In [8]:
gf.subclass_description

57     Food service activities of take away counters
58     Food service activities of take away counters
59     Food service activities of take away counters
60     Food service activities of take away counters
61     Food service activities of take away counters
62     Food service activities of take away counters
63     Food service activities of take away counters
64     Food service activities of take away counters
65     Food service activities of take away counters
66     Food service activities of take away counters
68     Food service activities of take away counters
69     Food service activities of take away counters
70     Food service activities of take away counters
72     Food service activities of take away counters
73     Food service activities of take away counters
74     Food service activities of take away counters
75     Food service activities of take away counters
76     Food service activities of take away counters
77     Food service activities of take away co

In [9]:
print(list(gf.companyname))

['KUNG FU KITCHEN', 'FISH AWAYS', 'MCD', 'DEBONAIRS', 'SAUSAGE SALOON', 'KFC', 'NANDOS', 'STEERS', 'SPUR', 'ROCOMAMAS', 'SIMPLY ASIA', 'DOMINOS PIZZA', 'BURGER KING', 'CHICKEN LICKEN', 'FOURNOS', 'MOCHACHOS', 'PAPACHINOS', 'PIZZA HUT', 'WIMPY', 'KRISPY KREME', 'KAUAI', 'COLCACCHIO', 'COMPASS GROUP', 'WIESENHOF', 'DOPPIO ZERO', "JIMMY'S KILLER", 'PAULS ICE-CREAM', 'WAKABERRY', 'SMOOCH', 'NU']


### Two Clients, ID=2248959, 4136719

In [10]:
neo4j_query="""
MATCH path = (m0:Merchant)<-[:TRANSACTED_AT]-(:Client)-[:TRANSACTED_AT]->(m1:Merchant) 
WHERE  ID(m0)<ID(m1) AND m1.triadic_56101_community=7916 AND m0.triadic_56101_community=7916 RETURN path LIMIT 5
"""
graph.run(neo4j_query).data()

[{'path': (_4683361)<-[:TRANSACTED_AT {db: 'CMProd202003', dbprefix: 'CMProd', period: '202003', transactionamount: -54.9, transactioncount: 1}]-(_2248959)-[:TRANSACTED_AT {db: 'CAProd202003', dbprefix: 'CAProd', period: '202003', transactionamount: -31.9, transactioncount: 1}]->(_4787417)},
 {'path': (_4683361)<-[:TRANSACTED_AT {db: 'CAProd202003', dbprefix: 'CAProd', period: '202003', transactionamount: -19.9, transactioncount: 1}]-(_4136719)-[:TRANSACTED_AT {db: 'CAProd202003', dbprefix: 'CAProd', period: '202003', transactionamount: -49.8, transactioncount: 2}]->(_4787417)},
 {'path': (_4683361)<-[:TRANSACTED_AT {db: 'CAProd202003', dbprefix: 'CAProd', period: '202003', transactionamount: -107.7, transactioncount: 1}]-(_3472932)-[:TRANSACTED_AT {db: 'CAProd202003', dbprefix: 'CAProd', period: '202003', transactionamount: -54.9, transactioncount: 1}]->(_4787417)},
 {'path': (_4829979)<-[:TRANSACTED_AT {db: 'SAProd202003', dbprefix: 'SAProd', period: '202003', transactionamount: -366

In [11]:
client_df_one=pd.DataFrame()
neo4j_query="""
MATCH (c:Client) 
WHERE  ID(c)=2248959 RETURN c
"""
s=graph.run(neo4j_query).data()

for d in s[0]['c']:
#     print(d)
    add_dict={'attribute':d, 'value':s[0]['c'][d]}
    client_df_one = client_df_one.append(add_dict, ignore_index=True)
client_df_one
client_df_one.rename(columns={'value':'client 1'}, inplace =True)

In [12]:
client_df_two=pd.DataFrame()
neo4j_query="""
MATCH (c:Client) 
WHERE  ID(c)=4136719 RETURN c
"""
s=graph.run(neo4j_query).data()

for d in s[0]['c']:
#     print(d)
    add_dict={'attribute':d, 'value':s[0]['c'][d]}
    client_df_two = client_df_two.append(add_dict, ignore_index=True)
client_df_two.rename(columns={'value':'client 2'}, inplace =True)

In [13]:
client_df_two

,attribute,client 2
0,period,202003
1,triadic_56101_community,7916
2,seg_l3_num,312
3,client_link_community_7916,1415
4,totaltransactionamount,-674.93
5,totaltransactioncount,10
6,dedupestatic,1.919770742e+11
7,client_link_community_6241,1336928
8,seg_l3_str,CB - PBCVM - Mass Market - Traditional


In [14]:
clients_df=client_df_one.merge(client_df_two,left_on='attribute',right_on='attribute')

In [15]:
print(clients_df.to_latex(index=False))

\begin{tabular}{lll}
\toprule
                  attribute &                              client 1 &                                client 2 \\
\midrule
                     period &                                202003 &                                  202003 \\
    triadic\_56101\_community &                                  7916 &                                    7916 \\
 client\_link\_community\_7916 &                                  1415 &                                    1415 \\
                 seg\_l3\_num &                                   324 &                                     312 \\
     totaltransactionamount &                              -5027.83 &                                 -674.93 \\
      totaltransactioncount &                                    30 &                                      10 \\
               dedupestatic &                       1.940665861e+11 &                         1.919770742e+11 \\
 client\_link\_community\_6241 &                  

### Two Merchants, ID=4683361, 4787417

In [16]:
merchant_one=pd.DataFrame()
neo4j_query="""
MATCH (m:Merchant) 
WHERE  ID(m)=4683361 RETURN m
"""
s=graph.run(neo4j_query).data()

In [17]:
for d in s[0]['m']:
#     print(d)
    add_dict={'attribute':d, 'value':s[0]['m'][d]}
    merchant_one = merchant_one.append(add_dict, ignore_index=True)
merchant_one
merchant_one.rename(columns={'value':'merchant 1'}, inplace =True)

In [18]:
merchant_two=pd.DataFrame()
neo4j_query="""
MATCH (m:Merchant) 
WHERE  ID(m)=4787417 RETURN m
"""
s=graph.run(neo4j_query).data()
for d in s[0]['m']:
#     print(d)
    add_dict={'attribute':d, 'value':s[0]['m'][d]}
    merchant_two = merchant_two.append(add_dict, ignore_index=True)
merchant_two
merchant_two.rename(columns={'value':'merchant 2'}, inplace =True)

In [19]:
merchant_two

,attribute,merchant 2
0,discretionary,2
1,period,202003
2,biadic community,7916
3,totaltransactionamount,-20780.8
4,subclass_id,56101
5,class_id,5610
6,channel,0
7,division_id,56
8,triadic_community,7242
9,franchisename,KFC MOLOTO


In [20]:
merchants_df=merchant_one.merge(merchant_two,left_on='attribute',right_on='attribute')

In [21]:
print(merchants_df.to_latex(index=False))

\begin{tabular}{lll}
\toprule
               attribute &      merchant 1 &  merchant 2 \\
\midrule
           discretionary &               2 &           2 \\
                  period &          202003 &      202003 \\
        biadic community &            7916 &        7916 \\
  totaltransactionamount &         -3509.3 &    -20780.8 \\
             subclass\_id &           56101 &       56101 \\
                class\_id &            5610 &        5610 \\
                 channel &               0 &           0 \\
             division\_id &              56 &          56 \\
       triadic\_community &            7242 &        7242 \\
           franchisename &  KFC PHOLA PARK &  KFC MOLOTO \\
        biadic\_community &            7916 &        7916 \\
            companyindex &              65 &          65 \\
 triadic\_56101\_community &            7916 &        7916 \\
                group\_id &             561 &         561 \\
             companyname &             KFC &         

In [22]:
neo4j_query="""MATCH (c:Client)-[tfl:TRANSACTED_AT]->(m:Merchant) WHERE ID(c)=2248959 AND ID(m)=4683361 RETURN tfl"""
s=graph.run(neo4j_query).data()

In [23]:
transaction_11=pd.DataFrame()
neo4j_query="""
MATCH (c:Client)-[tfl:TRANSACTED_AT]->(m:Merchant) WHERE ID(c)=2248959 AND ID(m)=4683361 RETURN tfl
"""
s=graph.run(neo4j_query).data()
for d in s[0]['tfl']:
#     print(d)
    add_dict={'attribute':d, 'value':s[0]['tfl'][d]}
    transaction_11 = transaction_11.append(add_dict, ignore_index=True)
# merchant_two
transaction_11.rename(columns={'value':'client 1 (1)'}, inplace =True)

In [24]:
transaction_12=pd.DataFrame()
neo4j_query="""
MATCH (c:Client)-[tfl:TRANSACTED_AT]->(m:Merchant) WHERE ID(c)=2248959 AND ID(m)=4787417 RETURN tfl
"""
s=graph.run(neo4j_query).data()
for d in s[0]['tfl']:
#     print(d)
    add_dict={'attribute':d, 'value':s[0]['tfl'][d]}
    transaction_12 = transaction_12.append(add_dict, ignore_index=True)
# merchant_two
transaction_12.rename(columns={'value':'client 1 (2)'}, inplace =True)

In [25]:
transaction_21=pd.DataFrame()
neo4j_query="""
MATCH (c:Client)-[tfl:TRANSACTED_AT]->(m:Merchant) WHERE ID(c)=4136719 AND ID(m)=4683361 RETURN tfl
"""
s=graph.run(neo4j_query).data()
for d in s[0]['tfl']:
#     print(d)
    add_dict={'attribute':d, 'value':s[0]['tfl'][d]}
    transaction_21 = transaction_21.append(add_dict, ignore_index=True)
# merchant_two
transaction_21.rename(columns={'value':'client 2 (1)'}, inplace =True)

In [26]:
transaction_22=pd.DataFrame()
neo4j_query="""
MATCH (c:Client)-[tfl:TRANSACTED_AT]->(m:Merchant) WHERE ID(c)=4136719 AND ID(m)=4787417 RETURN tfl
"""
s=graph.run(neo4j_query).data()
for d in s[0]['tfl']:
#     print(d)
    add_dict={'attribute':d, 'value':s[0]['tfl'][d]}
    transaction_22 = transaction_22.append(add_dict, ignore_index=True)
# merchant_two
transaction_22.rename(columns={'value':'client 2 (2)'}, inplace =True)

In [27]:
transaction_summary_df=transaction_11.merge(transaction_12, left_on='attribute',right_on='attribute')
transaction_summary_df=transaction_summary_df.merge(transaction_21, left_on='attribute',right_on='attribute')
transaction_summary_df=transaction_summary_df.merge(transaction_22, left_on='attribute',right_on='attribute')

In [28]:
print(transaction_summary_df.to_latex(index=False))

\begin{tabular}{lllll}
\toprule
         attribute &  client 1 (1) &  client 1 (2) &  client 2 (1) &  client 2 (2) \\
\midrule
            period &        202003 &        202003 &        202003 &        202003 \\
 transactionamount &         -54.9 &         -31.9 &         -19.9 &         -49.8 \\
          dbprefix &        CMProd &        CAProd &        CAProd &        CAProd \\
  transactioncount &             1 &             1 &             1 &             2 \\
                db &  CMProd202003 &  CAProd202003 &  CAProd202003 &  CAProd202003 \\
\bottomrule
\end{tabular}



In [29]:
neo4j_query=f"""
MATCH (c:Client) 
WHERE EXISTS(c.client_link_community_7916) 
RETURN COUNT(c.client_link_community_7916) as count, c.client_link_community_7916 as client_link_community
ORDER BY count ASC
"""
df=graph.run(neo4j_query).to_data_frame()

In [30]:
print(df.to_latex(index=False))

\begin{tabular}{rr}
\toprule
 count &  client\_link\_community \\
\midrule
    48 &                    822 \\
   108 &                    232 \\
   112 &                     44 \\
   146 &                     86 \\
   172 &                   1415 \\
   206 &                     27 \\
   237 &                    220 \\
   238 &                   1779 \\
   288 &                     95 \\
   333 &                     75 \\
   372 &                     31 \\
   429 &                    176 \\
\bottomrule
\end{tabular}



In [74]:
neo4j_query="""
MATCH (m:Merchant {subclass_id:56101})
WHERE EXISTS(m.triadic_56101_community) 
RETURN COUNT(m.triadic_56101_community) as membercount, m.triadic_56101_community as triadic_56101_community
ORDER BY membercount ASC;
"""

In [75]:
df=graph.run(neo4j_query).to_data_frame()

In [76]:
df

,membercount,triadic_56101_community
0,1,0
1,1,9724
2,1,1
3,1,9725
4,1,6
...,...,...
5127,351,5936
5128,364,6365
5129,398,8188
5130,517,1772


In [77]:
triadic_56101_community_list=list(df[df.membercount>20].triadic_56101_community)

In [78]:
print(df[df.membercount>20].to_latex(index=False))

\begin{tabular}{rr}
\toprule
 membercount &  triadic\_56101\_community \\
\midrule
          25 &                     7916 \\
          30 &                     2697 \\
          32 &                     6246 \\
          44 &                     6305 \\
          55 &                     6314 \\
          61 &                      290 \\
          69 &                     6790 \\
          72 &                     8577 \\
          84 &                     2160 \\
          96 &                     5959 \\
         109 &                     6244 \\
         115 &                     7805 \\
         122 &                     7579 \\
         128 &                     1875 \\
         149 &                     7544 \\
         187 &                     5898 \\
         241 &                     4914 \\
         258 &                      154 \\
         319 &                     6778 \\
         351 &                     5936 \\
         364 &                     6365 \\
         398 &

In [41]:
community=7916
df=pd.read_csv(filepath_or_buffer='/media/lnr-ai/christo/github_repos/graphing/data/triadic_56101_community_df_{community}'.format(community=community)) 

In [45]:
df.iloc[0].franchisename

'KFC GROBLERS'

In [43]:
print(df[['franchisename','score']].to_latex(index=False))

\begin{tabular}{lr}
\toprule
   franchisename &     score \\
\midrule
    KFC GROBLERS &  0.809422 \\
  NANDOS GROBLER &  0.670787 \\
    DEBONAIRS SI &  0.660889 \\
    KFC MOUTSIYA &  0.601046 \\
  WIMPY GROBLERS &  0.575624 \\
    KFC KWAGGAFO &  0.470889 \\
  STEERS MORATIW &  0.441075 \\
  DEBONAIRS GROB &  0.394640 \\
      KFC MOLOTO &  0.277500 \\
    McD Kwagga M &  0.267938 \\
    McD Groblers &  0.259299 \\
    KFC MOUTSE M &  0.232283 \\
  KFC MOUTSIYA M &  0.213750 \\
    KFC PHOLA PA &  0.195549 \\
  DEBONAIRS MORA &  0.186734 \\
  DEBONAIRS SIYA &  0.186734 \\
  DEBONAIRS JANE &  0.150000 \\
  KFC GROBLERSDA &  0.150000 \\
    KFC JANE FUR &  0.150000 \\
    KFC MARBLE H &  0.150000 \\
    KFC Moratiwa &  0.150000 \\
  KFC PHOLA PARK &  0.150000 \\
    KFC SIYABUSW &  0.150000 \\
   KFC SIYABUSWA &  0.150000 \\
 McD Groblersdal &  0.150000 \\
\bottomrule
\end{tabular}



In [56]:
# path='/media/lnr-ai/christo/github_repos/graphing/data'
path='/media/lnr-ai/christo/github_repos/graphing/data'
file_list=glob.glob(path+'/*')
df.append(ignore_index=)

In [68]:
answer_dict={}
answer_df=pd.DataFrame()
for file_name in file_list:
#     print(file_name)
    if 'triadic_56101' in file_name:
        print(file_name,int(file_name.split('_')[-1]))
        df=pd.read_csv(filepath_or_buffer=file_name)
        answer_dict['franchisename_1']=df.iloc[0].franchisename
        answer_dict['franchisename_2']=df.iloc[1].franchisename
        answer_dict['franchisename_3']=df.iloc[2].franchisename
        answer_dict['franchisename_4']=df.iloc[3].franchisename           
        answer_dict['community']=int(file_name.split('_')[-1])   
        answer_df=answer_df.append(answer_dict, ignore_index=True)

/media/lnr-ai/christo/github_repos/graphing/data/triadic_56101_community_df_1875 1875
/media/lnr-ai/christo/github_repos/graphing/data/triadic_56101_community_df_154 154
/media/lnr-ai/christo/github_repos/graphing/data/triadic_56101_community_df_1772 1772
/media/lnr-ai/christo/github_repos/graphing/data/triadic_56101_community_df_2160 2160
/media/lnr-ai/christo/github_repos/graphing/data/triadic_56101_community_df_2697 2697
/media/lnr-ai/christo/github_repos/graphing/data/triadic_56101_community_df_290 290
/media/lnr-ai/christo/github_repos/graphing/data/triadic_56101_community_df_4914 4914
/media/lnr-ai/christo/github_repos/graphing/data/triadic_56101_community_df_5898 5898
/media/lnr-ai/christo/github_repos/graphing/data/triadic_56101_community_df_5936 5936
/media/lnr-ai/christo/github_repos/graphing/data/triadic_56101_community_df_5959 5959
/media/lnr-ai/christo/github_repos/graphing/data/triadic_56101_community_df_5989 5989
/media/lnr-ai/christo/github_repos/graphing/data/triadic_5

In [69]:
answer_df

,community,franchisename_1,franchisename_2,franchisename_3,franchisename_4
0,1875.0,NANDOS MAKHA,SPUR AMARILL,Nandos Paledi,KFC MASINGIT
1,154.0,BURGER KING JE,KFC BLUE VAL,SIMPLY ASIA MA,KFC VORNA VA
2,1772.0,MCD MountEdgec,KFC PHILANI VA,STEERS DINER U,WIMPY PINECR
3,2160.0,WIESENHOF COSM,KFC TASBET P,STEERS MIDDE,KFC MIDDELBU
4,2697.0,MCD Phuthaditj,KENTUCKY FRIED,KFC BETHLEHE,WIMPY NEW MA
5,290.0,KFC IZINGOLWEN,MC DONALDS L,STEERS WILDCOA,WIMPY SHELLY
6,4914.0,KEI STEERS D,KFC GREENFIE,Nandos Circus,DEBONAIRS BT N
7,5898.0,WIMPY METLIF,McDONALDS LINT,KFC KABEGA P,KFC WALKER D
8,5936.0,WIMPY NEW RE,KFC ELDORADO,WIMPY THE GL,MCD Protea Gle
9,5959.0,KFC NELSPRUI,WIMPY BARBERTO,NANDOS NELSPR,WIESENHOF MA


In [70]:
location_df=pd.DataFrame()
d={'community':1875, 'area':'Makhado', 'location': '-23.040329, 29.907955'}
location_df=location_df.append(d, ignore_index=True)
d={'community':154, 'area':'Centurion', 'location': '-25.841164, 28.185680'}
location_df=location_df.append(d, ignore_index=True)
d={'community':1772, 'area':'Umhlangha', 'location': '-29.725526, 31.038057'}
location_df=location_df.append(d, ignore_index=True)
d={'community':2160, 'area':'Emalahleni', 'location': '-25.883386, 29.233104'}
location_df=location_df.append(d, ignore_index=True)
d={'community':2697, 'area':'Bethlehem', 'location': '-28.182308, 28.304568'}
location_df=location_df.append(d, ignore_index=True)
d={'community':290, 'area':'Port Edward', 'location': '30.784865, 30.133764'}
location_df=location_df.append(d, ignore_index=True)
d={'community':4914, 'area':'East London', 'location': '-32.505660, 27.980685'}
location_df=location_df.append(d, ignore_index=True)
d={'community':5898, 'area':'Port Elizabeth', 'location': '-33.946310, 25.509027'}
location_df=location_df.append(d, ignore_index=True)
d={'community':5936, 'area':'Alberton', 'location': '-26.261012, 28.115685'}
location_df=location_df.append(d, ignore_index=True)
d={'community':5959, 'area':'Nelspruit', 'location': '-25.346253, 30.962704'}
location_df=location_df.append(d, ignore_index=True)
d={'community':5989, 'area':'Cape Town', 'location': '-33.958561, 18.539641'}
location_df=location_df.append(d, ignore_index=True)
d={'community':6244, 'area':'Bloemfontein', 'location': '-29.086566, 26.211717'}
location_df=location_df.append(d, ignore_index=True)
d={'community':6246, 'area':'Ventersdorp', 'location': '-26.332893, 26.804443'}
location_df=location_df.append(d, ignore_index=True)
d={'community':6305, 'area':'Worcester', 'location': '-33.641490, 19.449015'}
location_df=location_df.append(d, ignore_index=True)
d={'community':6314, 'area':'Klerksdorp', 'location': '-26.866751, 26.643835'}
location_df=location_df.append(d, ignore_index=True)
d={'community':6365, 'area':'Kempton Park', 'location': '-26.062117, 28.220977'}
location_df=location_df.append(d, ignore_index=True)
d={'community':6778, 'area':'Wonderboom', 'location': '-25.685555, 28.190614'}
location_df=location_df.append(d, ignore_index=True)
d={'community':6790, 'area':'Kimberley', 'location': '-28.756451, 24.753912'}
location_df=location_df.append(d, ignore_index=True)
d={'community':7544, 'area':'Eshowe', 'location': '-28.894043, 31.467636'}
location_df=location_df.append(d, ignore_index=True)
d={'community':7579, 'area':'Rustenburg', 'location': '-25.361360, 27.099350'}
location_df=location_df.append(d, ignore_index=True)
d={'community':7805, 'area':'Pietermaritzburg', 'location': '-29.615721, 30.393890'}
location_df=location_df.append(d, ignore_index=True)
d={'community':7916, 'area':'Groblersdal', 'location': '-25.168414, 29.396050'}
location_df=location_df.append(d, ignore_index=True)
d={'community':8188, 'area':'Florida', 'location': '-26.147890, 27.901389'}
location_df=location_df.append(d, ignore_index=True)
d={'community':8577, 'area':'Vanderbijlpark', 'location': '-26.709675, 27.822882'}
location_df=location_df.append(d, ignore_index=True)

In [73]:
df

,franchisename,score,community,membercount
0,MCDONALD'S V,2.011277,8577,72
1,STEERS VAAL MA,1.512435,8577,72
2,KFC Vanderbijl,1.184150,8577,72
3,WIMPY VAALMA,1.079826,8577,72
4,NANDOS VANDE,0.865251,8577,72
...,...,...,...,...
67,SPUR CUNCUN,0.150000,8577,72
68,SPUR OAK CREEK,0.150000,8577,72
69,STEERS MEYERTON,0.150000,8577,72
70,WIMPY RIVER SQUARE,0.150000,8577,72


In [82]:
gf=df.merge(answer_df, left_on='triadic_56101_community', right_on='community')
result_df=location_df.merge(gf, left_on='community',right_on='community')

In [83]:
result_df.to_csv('/media/lnr-ai/christo/github_repos/graphing/data/fastfood_community_df.csv', index=False)

In [87]:
print(result_df[['community', 'area', 'membercount','franchisename_1']].sort_values('membercount').to_latex(index=False))

\begin{tabular}{rlrl}
\toprule
 community &              area &  membercount & franchisename\_1 \\
\midrule
    7916.0 &       Groblersdal &           25 &    KFC GROBLERS \\
    2697.0 &         Bethlehem &           30 &  MCD Phuthaditj \\
    6246.0 &       Ventersdorp &           32 &    STEERS VENTE \\
    6305.0 &         Worcester &           44 &  STEERS RESTAUR \\
    6314.0 &        Klerksdorp &           55 &    WIMPY GOUDKO \\
     290.0 &       Port Edward &           61 &  KFC IZINGOLWEN \\
    6790.0 &         Kimberley &           69 &    KAUAI KIMBER \\
    8577.0 &    Vanderbijlpark &           72 &    MCDONALD'S V \\
    2160.0 &        Emalahleni &           84 &  WIESENHOF COSM \\
    5959.0 &         Nelspruit &           96 &    KFC NELSPRUI \\
    6244.0 &      Bloemfontein &          109 &    STEERS PRELL \\
    7805.0 &  Pietermaritzburg &          115 &    MCDONALD'S C \\
    7579.0 &        Rustenburg &          122 &    KFC SUN VILL \\
    1875.0 &         

In [51]:
# this code takes the content of "text.txt" and store it into my_text
extension="*.md"
search_word="docker"
# path='/media/lnr-ai/christo/github_repos/money_personalities/'
result_dict={}
path='/media/lnr-ai/christo/github_repos/graphing/data/'
file_list=glob.glob(path+'/'+extension)
#     print(path+extension)
for file_name in file_list:
    print(file_name)
    with open(file_name) as file:
        my_text = file.read()
        if my_text.count(search_word)>0:
            print('found {search_word} in {file_name}!'.format(search_word=search_word, file_name=file_name))
            result_dict[file_name]=my_text.count(search_word)

['/media/lnr-ai/christo/github_repos/graphing/data']

In [ ]:
graph = Graph("bolt://localhost:7687", user='neo4j', password='newPassword')
hf=pd.DataFrame()
donelist=[]
leftoverlist=triadic_56101_community_list.copy()
counter=0
for community in triadic_56101_community_list:
    print('Doing community {community}, {counter} of {total}'.format(community=community,counter=len(donelist),total=len(triadic_56101_community_list)))
    graphname="""fastfood_{community}""".format(community=community)
    query="""CALL gds.graph.exists('{graphname}') YIELD exists;""".format(graphname=graphname)
    exists=graph.run(query).data()
    if exists[0]['exists']:
        print("graph exists!")
        graph.run("""CALL gds.graph.drop('{graphname}') YIELD graphName;""".format(graphname=graphname)).data()
    #     CALL gds.graph.drop('my-store-graph') YIELD graphName;
    else:
        print("graph doesn't exist!")
    # ----------------------------------------------------------------------------
    create_graph_query="""
    CALL gds.graph.create.cypher(
        'fastfood_{community}',
        'MATCH (m:Merchant {{subclass_id:56101}}) WHERE EXISTS(m.triadic_56101_community) AND 
        m.triadic_56101_community = {community} RETURN ID(m) AS id',
        'MATCH (a:Merchant)-[:TRIADIC_MERCHANT_FEET_LINK]->(b:Merchant) WHERE ID(a)<ID(b) AND
        EXISTS(a.triadic_56101_community) AND EXISTS(b.triadic_56101_community) AND
        a.triadic_56101_community = {community} AND
        b.triadic_56101_community = {community}
         RETURN ID(a) AS source, ID(b) AS target'
    );
    """.format(community=community)
    graph.run(create_graph_query).data()

    pagerank_query="""CALL gds.pageRank.stream('{graphname}', {{ maxIterations: 20, dampingFactor: 0.85 }})
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).franchisename AS franchisename, score
    ORDER BY score DESC, franchisename ASC;""".format(graphname=graphname)
    gf=graph.run(pagerank_query).to_data_frame()
    gf['community']=community
    gf['membercount']=list(df[df.triadic_56101_community==community]['membercount'])[0]
    gf.to_csv(path_or_buf='/media/lnr-ai/christo/github_repos/graphing/data/triadic_56101_community_df_{community}'.format(community=community),index=False) 
    hf = hf.append(gf, ignore_index=True)
    print(gf)
    print("Delete graph {graphname}...".format(graphname=graphname))
    graph.run("""CALL gds.graph.drop('{graphname}') YIELD graphName;""".format(graphname=graphname)).data()
#     print('-Sleep 60s---------------------------------')
#     time.sleep(60)
    donelist.append(community)
    leftoverlist.pop(leftoverlist.index(community))
    print(leftoverlist)
    counter+=1
    print("====================================================================================")

In [ ]:
leftoverlist

In [ ]:
# from py2neo import Graph
graph = Graph("bolt://localhost:7687", user='neo4j', password='newPassword')
hf=pd.DataFrame()
donelist=[]
tmp_list=leftoverlist.copy()
# leftoverlist=tmp_list
counter=0
for community in leftoverlist:
    print('Doing community {community}, {counter} of {total}'.format(community=community,counter=len(donelist),total=len(tmp_list)))
    graphname="""fastfood_{community}""".format(community=community)
    query="""CALL gds.graph.exists('{graphname}') YIELD exists;""".format(graphname=graphname)
    exists=graph.run(query).data()
    if exists[0]['exists']:
        print("graph exists!")
        graph.run("""CALL gds.graph.drop('{graphname}') YIELD graphName;""".format(graphname=graphname)).data()
    #     CALL gds.graph.drop('my-store-graph') YIELD graphName;
    else:
        print("graph doesn't exist!")
    # ----------------------------------------------------------------------------
    create_graph_query="""
    CALL gds.graph.create.cypher(
        'fastfood_{community}',
        'MATCH (m:Merchant {{subclass_id:56101}}) WHERE EXISTS(m.triadic_56101_community) AND 
        m.triadic_56101_community = {community} RETURN ID(m) AS id',
        'MATCH (a:Merchant)-[:TRIADIC_MERCHANT_FEET_LINK]->(b:Merchant) WHERE ID(a)<ID(b) AND
        EXISTS(a.triadic_56101_community) AND EXISTS(b.triadic_56101_community) AND
        a.triadic_56101_community = {community} AND
        b.triadic_56101_community = {community}
         RETURN ID(a) AS source, ID(b) AS target'
    );
    """.format(community=community)
    graph.run(create_graph_query).data()

    pagerank_query="""CALL gds.pageRank.stream('{graphname}', {{ maxIterations: 20, dampingFactor: 0.85 }})
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).franchisename AS franchisename, score
    ORDER BY score DESC, franchisename ASC;""".format(graphname=graphname)
    gf=graph.run(pagerank_query).to_data_frame()
    gf['community']=community
    gf['membercount']=list(df[df.triadic_56101_community==community]['membercount'])[0]
    gf.to_csv(path_or_buf='/media/lnr-ai/christo/github_repos/graphing/data/triadic_56101_community_df_{community}'.format(community=community),index=False) 
    hf = hf.append(gf, ignore_index=True)
    print(gf)
    print("Delete graph {graphname}...".format(graphname=graphname))
    graph.run("""CALL gds.graph.drop('{graphname}') YIELD graphName;""".format(graphname=graphname)).data()
#     print('-Sleep 60s---------------------------------')
#     time.sleep(60)
    donelist.append(community)
    tmp_list.pop(tmp_list.index(community))
    print(tmp_list)
    counter+=1
    print("====================================================================================")

In [ ]:
# from py2neo import Graph
graph = Graph("bolt://localhost:7687", user='neo4j', password='newPassword')
hf=pd.DataFrame()
donelist=[]
leftoverlist=tmp_list.copy()
# leftoverlist=tmp_list
counter=0
for community in leftoverlist:
    print('Doing community {community}, {counter} of {total}'.format(community=community,counter=len(donelist),total=len(tmp_list)))
    graphname="""fastfood_{community}""".format(community=community)
    query="""CALL gds.graph.exists('{graphname}') YIELD exists;""".format(graphname=graphname)
    exists=graph.run(query).data()
    if exists[0]['exists']:
        print("graph exists!")
        graph.run("""CALL gds.graph.drop('{graphname}') YIELD graphName;""".format(graphname=graphname)).data()
    #     CALL gds.graph.drop('my-store-graph') YIELD graphName;
    else:
        print("graph doesn't exist!")
    # ----------------------------------------------------------------------------
    create_graph_query="""
    CALL gds.graph.create.cypher(
        'fastfood_{community}',
        'MATCH (m:Merchant {{subclass_id:56101}}) WHERE EXISTS(m.triadic_56101_community) AND 
        m.triadic_56101_community = {community} RETURN ID(m) AS id',
        'MATCH (a:Merchant)-[:TRIADIC_MERCHANT_FEET_LINK]->(b:Merchant) WHERE ID(a)<ID(b) AND
        EXISTS(a.triadic_56101_community) AND EXISTS(b.triadic_56101_community) AND
        a.triadic_56101_community = {community} AND
        b.triadic_56101_community = {community}
         RETURN ID(a) AS source, ID(b) AS target'
    );
    """.format(community=community)
    graph.run(create_graph_query).data()

    pagerank_query="""CALL gds.pageRank.stream('{graphname}', {{ maxIterations: 20, dampingFactor: 0.85 }})
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).franchisename AS franchisename, score
    ORDER BY score DESC, franchisename ASC;""".format(graphname=graphname)
    gf=graph.run(pagerank_query).to_data_frame()
    gf['community']=community
    gf['membercount']=list(df[df.triadic_56101_community==community]['membercount'])[0]
    gf.to_csv(path_or_buf='/media/lnr-ai/christo/github_repos/graphing/data/triadic_56101_community_df_{community}'.format(community=community),index=False) 
    hf = hf.append(gf, ignore_index=True)
    print(gf)
    print("Delete graph {graphname}...".format(graphname=graphname))
    graph.run("""CALL gds.graph.drop('{graphname}') YIELD graphName;""".format(graphname=graphname)).data()
#     print('-Sleep 60s---------------------------------')
#     time.sleep(60)
    donelist.append(community)
    tmp_list.pop(tmp_list.index(community))
    print(tmp_list)
    counter+=1
    print("====================================================================================")

In [ ]:
# graph.run("""CALL gds.graph.exists('fastfood_{community}') YIELD exists;""".format(community=community)).data

In [ ]:
graph.run("""CALL gds.graph.exists('fastfood_{community}');""".format(community=community)).data

In [ ]:
community=5936
print('Doing community {community}'.format(community=community))
graphname="""fastfood_{community}""".format(community=community)
query="""CALL gds.graph.exists('{graphname}') YIELD exists;""".format(graphname=graphname)
exists=graph.run(query).data()
if exists[0]['exists']:
    print("graph exists!")
    graph.run("""CALL gds.graph.drop('{graphname}') YIELD graphName;""".format(graphname=graphname)).data()
#     CALL gds.graph.drop('my-store-graph') YIELD graphName;
else:
    print("graph doesn't exist!")
# ----------------------------------------------------------------------------
create_graph_query="""
CALL gds.graph.create.cypher(
    'fastfood_{community}',
    'MATCH (m:Merchant {{subclass_id:56101}}) WHERE EXISTS(m.triadic_56101_community) AND 
    m.triadic_56101_community = {community} RETURN ID(m) AS id',
    'MATCH (a:Merchant)-[:TRIADIC_MERCHANT_FEET_LINK]->(b:Merchant) WHERE ID(a)<ID(b) AND
    EXISTS(a.triadic_56101_community) AND EXISTS(b.triadic_56101_community) AND
    a.triadic_56101_community = {community} AND
    b.triadic_56101_community = {community}
     RETURN ID(a) AS source, ID(b) AS target'
);
""".format(community=community)
graph.run(create_graph_query).data()

pagerank_query="""CALL gds.pageRank.stream('{graphname}', {{ maxIterations: 20, dampingFactor: 0.85 }})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).franchisename AS franchisename, score
ORDER BY score DESC, franchisename ASC;""".format(graphname=graphname)
gf=graph.run(pagerank_query).to_data_frame()
gf['community']=community
gf['membercount']=list(df[df.triadic_56101_community==community]['membercount'])[0]
hf = hf.append(gf, ignore_index=True)
print(gf)
print("Delete graph {graphname}...".format(graphname=graphname))
graph.run("""CALL gds.graph.drop('{graphname}') YIELD graphName;""".format(graphname=graphname)).data()

print("====================================================================================")

In [ ]:
triadic_56101_community_list

### Delete community client link edges

Choose community:

In [ ]:
community=2697

In [ ]:
graph = Graph("bolt://localhost:7687", user='neo4j', password='newPassword')
delete_query="""CALL apoc.periodic.iterate('MATCH (client0:Client)-[cl:CLIENT_LINK_{community}]-(client1:Client)
WHERE ID(client1)<ID(client0)
RETURN cl;'
, 'DELETE cl', {{batchSize:500, iterateList:true, parallel:true}})
yield batches, total return batches, total;""".format(community=community)
graph.run(delete_query).data()

In [ ]:
graph = Graph("bolt://localhost:7687", user='neo4j', password='newPassword')
count_links_query="""MATCH (client0:Client)-[cl:CLIENT_LINK_{community}]-(client1:Client)
WHERE ID(client1)<ID(client0)
RETURN count(cl) as client_links;""".format(community=community)
# graph.run(count_links_query).data()
df=graph.run(count_links_query).to_data_frame()

In [ ]:
df

In [ ]:
# community=2697
graph = Graph("bolt://localhost:7687", user='neo4j', password='newPassword')
client_link_query="""CALL apoc.periodic.iterate('MATCH (client0:Client)-[t0:TRANSACTED_AT]->(merchant:Merchant)<-[t1:TRANSACTED_AT]-(client1:Client)
WHERE ID(client0) < ID(client1) AND 
merchant.triadic_56101_community = {community} 
RETURN client0, client1, t0, t1;
', "MERGE (client0)-[link:CLIENT_LINK_{community}]-(client1)
ON CREATE SET link.count = 1,
link.ID0=ID(client0),
link.ID1=ID(client1),
link.transactioncount0 = t0.transactioncount,
link.transactioncount1 = t1.transactioncount,
link.transactionamount0 = t0.transactionamount,
link.transactionamount1 = t1.transactionamount
ON MATCH SET link.count = link.count + 1,
link.transactioncount0 = link.transactioncount0+t0.transactioncount,
link.transactioncount1 = link.transactioncount1+t1.transactioncount,
link.transactionamount0 = link.transactionamount0+t0.transactionamount,
link.transactionamount1 = link.transactionamount1+t1.transactionamount;", {{batchSize:500, iterateList:true, parallel:true}})
yield batches, total return batches, total;
""".format(community=community)
graph.run(client_link_query).data()

### Check sub graph existence and delete

In [ ]:
print('Doing community {community}'.format(community=community))
graphname="""client_link_graph_{community}""".format(community=community)
query="""CALL gds.graph.exists('{graphname}') YIELD exists;""".format(graphname=graphname)
exists=graph.run(query).data()
if exists[0]['exists']:
    print("graph exists!")
    graph.run("""CALL gds.graph.drop('{graphname}') YIELD graphName;""".format(graphname=graphname)).data()
#     CALL gds.graph.drop('my-store-graph') YIELD graphName;
else:
    print("graph doesn't exist!")

### Create a subgraph for clients related via a client_link relationship between clients for ONE merchant cluster/community

In [ ]:
subgraph_query="""CALL gds.graph.create.cypher(
    '{graphname}',
    'MATCH (c0:Client)-[:TRANSACTED_AT]->(m:Merchant) WHERE EXISTS(m.triadic_56101_community) AND 
    m.triadic_56101_community = {community} RETURN id(c0) AS id',
    'MATCH (c0:Client)-[rel:CLIENT_LINK_{community}]-(c1:Client) RETURN id(c0) AS source, id(c1) AS target, rel.count as count'
)
YIELD graphName, nodeCount, relationshipCount, createMillis;""".format(graphname=graphname,community=community)
graph.run(subgraph_query).data()

### Cluster this subgraph of clients and STREAM output

In [ ]:
louvain_stream_query="""CALL gds.louvain.stream('client_link_graph_{community}', {{ relationshipWeightProperty: 'count', 
maxLevels:10, 
maxIterations:200, includeIntermediateCommunities:true}})
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).dedupestatic AS dedupestatic, communityId, intermediateCommunityIds
ORDER BY communityId DESC;""".format(community=community)
# graph.run(louvain_stream_query).data()
df=graph.run(louvain_stream_query).to_data_frame()

In [ ]:
df

### Cluster this  subgraph of clients and WRITE output on each client node

In [ ]:
louvain_write_query="""CALL gds.louvain.write('client_link_graph_{community}', {{ relationshipWeightProperty: 'count', 
maxLevels:10, 
maxIterations:200, writeProperty: 'client_link_community_{community}' }})
YIELD communityCount, modularity, modularities""".format(community=community)
graph.run(louvain_write_query).data()

### Inspect client membership of merhcant community

In [ ]:
client_membership_query="""MATCH (c:Client) 
WHERE EXISTS(c.client_link_community_{community}) 
RETURN COUNT(c.client_link_community_{community}) as count, c.client_link_community_{community} as 
client_link_community
ORDER BY count ASC""".format(community=community)
# graph.run(louvain_stream_query).data()
df=graph.run(client_membership_query).to_data_frame()